In [1]:
import pandas as pd
from datetime import datetime, timedelta,date

In [2]:
pd.set_option('display.max_rows',100)
pd.set_option('display.max_columns',100)

In [3]:
df = pd.read_csv('Orig_data.csv',error_bad_lines=False)
df.head()

,Channel,Date,DCM Campaign Name,Market,GEP,KPI,Site,Ad Type,DCM Placement ID,Placement Name,Publisher Impressions,Publisher Clicks,Net Media Cost,Currency,Lead Form Ad Leads
0,Social,09/09/2018,DEU_gaPMG_AZR_AZR_Q1_19_Trial-Social- 39684,Germany,AZR,Trial,Facebook,Canvas Ad,224009494,Facebook»DEV-ITP-AZR-FB|CanvasAds|Interesttarg...,8009.0,38.0,134.53,EUR,NaN
1,Social,10/09/2018,DEU_gaPMG_AZR_AZR_Q1_19_Trial-Social- 39684,Germany,AZR,Trial,Facebook,Canvas Ad,224009494,Facebook»DEV-ITP-AZR-FB|CanvasAds|Interesttarg...,20102.0,44.0,145.47,EUR,NaN
2,Social,11/09/2018,DEU_gaPMG_AZR_AZR_Q1_19_Trial-Social- 39684,Germany,AZR,Trial,Facebook,Canvas Ad,224009494,Facebook»DEV-ITP-AZR-FB|CanvasAds|Interesttarg...,20832.0,53.0,199.72,EUR,NaN
3,Social,12/09/2018,DEU_gaPMG_AZR_AZR_Q1_19_Trial-Social- 39684,Germany,AZR,Trial,Facebook,Canvas Ad,224009494,Facebook»DEV-ITP-AZR-FB|CanvasAds|Interesttarg...,11509.0,30.0,111.20,EUR,NaN
4,Social,13/09/2018,DEU_gaPMG_AZR_AZR_Q1_19_Trial-Social- 39684,Germany,AZR,Trial,Facebook,Canvas Ad,224009494,Facebook»DEV-ITP-AZR-FB|CanvasAds|Interesttarg...,13274.0,31.0,110.62,EUR,NaN


In [4]:
df.columns

Index(['Channel', 'Date', 'DCM Campaign Name', 'Market', 'GEP', 'KPI', 'Site',
       'Ad Type', 'DCM Placement ID', 'Placement Name',
       'Publisher Impressions', 'Publisher Clicks', 'Net Media Cost',
       'Currency', 'Lead Form Ad Leads'],
      dtype='object')

# Make a new dataframe

In [5]:
columns = ['Lookup Key 1','Lookup Key 2', 'Source', 'Weekend Date','Clean Market','Clean Ad Type','Campaign Name ID','Campaign ID','Campaign External ID','Clean Publisher','Site ID','Creative Name','Creative ID']
new_df = pd.DataFrame(columns=columns)
new_df

,Lookup Key 1,Lookup Key 2,Source,Weekend Date,Clean Market,Clean Ad Type,Campaign Name ID,Campaign ID,Campaign External ID,Clean Publisher,Site ID,Creative Name,Creative ID


In [6]:
def date_to_key(date_row):
    return str((datetime.strptime(date_row, "%d/%m/%Y").date()- date(1900,1,1)).days + 2)

In [7]:
new_df['Lookup Key 1'] = df['Date'].apply(date_to_key) + df['DCM Placement ID'].astype(str) 
new_df['Lookup Key 2'] = df['Date'].apply(date_to_key) + df['Placement Name'].astype(str) 

In [8]:
new_df['Source'] = "Appended from Pub-Facebook"

In [9]:
def get_weekend_date(date):
    return (datetime.strptime(date, "%d/%m/%Y").date() + timedelta(6 - datetime.strptime(date, "%d/%m/%Y").date().weekday())).strftime('%d-%m-%Y')

new_df['Weekend Date'] = df['Date'].apply(get_weekend_date)

In [10]:
new_df['Clean Publisher'] = "Look at taxonomy"
new_df.loc[(df.apply(lambda x: "FACEBOOK" in x['Placement Name'].upper(), axis=1)),"Clean Publisher"] = "Facebook"
new_df.loc[(df.apply(lambda x: "INSTAGRAM" in x['Placement Name'].upper(), axis=1)),"Clean Publisher"] = "Facebook-Instagram"

In [11]:
new_df['Site ID']  = "Not Facebook"
new_df.loc[new_df['Clean Publisher']=="Facebook", 'Site ID'] = "3654128"

In [12]:
new_df['Clean Ad Type'] = "Not filled properly"
new_df.loc[(df.apply(lambda x: "canvas" in x['Ad Type'].lower(), axis=1)),"Clean Ad Type"] = "Link"
new_df.loc[(df.apply(lambda x: "link" in x['Ad Type'].lower(), axis=1)),"Clean Ad Type"] = "Link"

In [13]:
key = pd.read_csv('Key.csv',skiprows=4)
key = key.drop(['Unnamed: 0','Unnamed: 4','Unnamed: 7','Unnamed: 10'],axis=1)
key.head()

,Market,Clean Market,Region,Ad Type,Clean Ad Type,KPI,Clean KPI,Site,Clean Publisher
0,AU,Australia,APAC,Link,Link,Leads,Lead,LinkedIn,LinkedIn
1,AUS,Australia,APAC,Link Ad,Link,CPC,Lead,Facebook,Facebook
2,Australia,Australia,APAC,Link Ads,Link,Lead,Lead,Facebook Inc,Facebook
3,CA,Canada,North America,Lead,Lead,NaN,NaN,Facebook Lead Form,Facebook
4,CAN,Canada,North America,Lead Ad,Lead,NaN,NaN,NaN,NaN


In [14]:
market_dict = pd.Series(key['Clean Market'].values,index=key.Market.values).to_dict()
new_df['Clean Market'] = df['Market']
new_df['Clean Market'].replace(market_dict, inplace=True)

In [15]:
new_df.head()

,Lookup Key 1,Lookup Key 2,Source,Weekend Date,Clean Market,Clean Ad Type,Campaign Name ID,Campaign ID,Campaign External ID,Clean Publisher,Site ID,Creative Name,Creative ID
0,43352224009494,43352Facebook»DEV-ITP-AZR-FB|CanvasAds|Interes...,Appended from Pub-Facebook,09-09-2018,Germany,Link,NaN,NaN,NaN,Facebook,3654128,NaN,NaN
1,43353224009494,43353Facebook»DEV-ITP-AZR-FB|CanvasAds|Interes...,Appended from Pub-Facebook,16-09-2018,Germany,Link,NaN,NaN,NaN,Facebook,3654128,NaN,NaN
2,43354224009494,43354Facebook»DEV-ITP-AZR-FB|CanvasAds|Interes...,Appended from Pub-Facebook,16-09-2018,Germany,Link,NaN,NaN,NaN,Facebook,3654128,NaN,NaN
3,43355224009494,43355Facebook»DEV-ITP-AZR-FB|CanvasAds|Interes...,Appended from Pub-Facebook,16-09-2018,Germany,Link,NaN,NaN,NaN,Facebook,3654128,NaN,NaN
4,43356224009494,43356Facebook»DEV-ITP-AZR-FB|CanvasAds|Interes...,Appended from Pub-Facebook,16-09-2018,Germany,Link,NaN,NaN,NaN,Facebook,3654128,NaN,NaN


In [16]:
lookup = pd.read_csv('Lookup.csv',encoding='latin1')
lookup.head()

,Placement Name,Placement ID,Campaign Name,Campaign ID,Campaign External ID,Publisher,Site ID,Creative Name,Creative ID
0,Facebook»AZP-Canvas-Interest&Role-Trial»1x1»Ca...,223298805,JPN_gaPMG_AZR_AZR_Q1_19_Trial-Social,21276694,720195,Facebook,3654128,AzurePlatform_FBCTCcanvaspage-BuyNowCTA-Img3_J...,102630373
1,Facebook»AZP-Canvas-Interest&Role-Trial»1x1»Ca...,223298805,JPN_gaPMG_AZR_AZR_Q1_19_Trial-Social,21276694,720195,Facebook,3654128,AzurePlatform_FBCTCcanvaspage-StartFreeCTA-Img...,102609621
2,Facebook»AZP-Canvas-Look a Like-Trial»1x1»Canv...,223297413,JPN_gaPMG_AZR_AZR_Q1_19_Trial-Social,21276694,720195,Facebook,3654128,AzurePlatform_FBCTCcanvaspage-StartFreeCTA-Img...,102609621
3,Facebook»Audience & Interest Targeting»1x1»Can...,224639151,AUS_gaPMG_AZR_AZR_Q1_19_Trial-Social,21358568,720191,Facebook,3654128,AzurePlatform_FBCTCcanvaspage-BuyNowCTA-Img3_A...,103316832
4,Facebook»Audience & Interest Targeting»1x1»Can...,224639151,AUS_gaPMG_AZR_AZR_Q1_19_Trial-Social,21358568,720191,Facebook,3654128,AzurePlatform_FBCTCcanvaspage-StartFreeCTA-Img...,103316829


In [17]:
new_df['Campaign Name ID'] = df['DCM Placement ID']
lookup_Campaign_dict = pd.Series(lookup['Campaign Name'].values,index=lookup['Placement ID'].values).to_dict()
new_df['Campaign Name ID'].replace(lookup_Campaign_dict, inplace=True)

In [18]:
new_df['Campaign ID'] = new_df['Campaign Name ID']
lookup_Campaign_id_dict = pd.Series(lookup['Campaign ID'].values,index=lookup['Campaign Name'].values).to_dict()
new_df['Campaign ID'].replace(lookup_Campaign_id_dict,inplace=True)

In [19]:
new_df['Campaign External ID'] = new_df['Campaign Name ID']
lookup_Campaign_external_id_dict = pd.Series(lookup['Campaign External ID'].values,index=lookup['Campaign Name'].values).to_dict()
new_df['Campaign External ID'].replace(lookup_Campaign_external_id_dict,inplace=True)

In [20]:
new_df['Creative Name'] = new_df['Campaign Name ID']
lookup_Creative_name_dict = pd.Series(lookup['Creative Name'].values,index=lookup['Campaign Name'].values).to_dict()
new_df['Creative Name'].replace(lookup_Creative_name_dict,inplace=True)

In [21]:
new_df['Creative ID'] = new_df['Campaign Name ID']
lookup_Creative_id_dict = pd.Series(lookup['Creative ID'].values,index=lookup['Campaign Name'].values).to_dict()
new_df['Creative ID'].replace(lookup_Creative_id_dict,inplace=True)

In [22]:
new_df.head()

,Lookup Key 1,Lookup Key 2,Source,Weekend Date,Clean Market,Clean Ad Type,Campaign Name ID,Campaign ID,Campaign External ID,Clean Publisher,Site ID,Creative Name,Creative ID
0,43352224009494,43352Facebook»DEV-ITP-AZR-FB|CanvasAds|Interes...,Appended from Pub-Facebook,09-09-2018,Germany,Link,DEU_gaPMG_AZR_AZR_Q1_19_Trial-Social- 39684,21359477,720194,Facebook,3654128,AzurePlatform_FBCTCBrandinMotion-BuyNow_DEU_12...,103350059
1,43353224009494,43353Facebook»DEV-ITP-AZR-FB|CanvasAds|Interes...,Appended from Pub-Facebook,16-09-2018,Germany,Link,DEU_gaPMG_AZR_AZR_Q1_19_Trial-Social- 39684,21359477,720194,Facebook,3654128,AzurePlatform_FBCTCBrandinMotion-BuyNow_DEU_12...,103350059
2,43354224009494,43354Facebook»DEV-ITP-AZR-FB|CanvasAds|Interes...,Appended from Pub-Facebook,16-09-2018,Germany,Link,DEU_gaPMG_AZR_AZR_Q1_19_Trial-Social- 39684,21359477,720194,Facebook,3654128,AzurePlatform_FBCTCBrandinMotion-BuyNow_DEU_12...,103350059
3,43355224009494,43355Facebook»DEV-ITP-AZR-FB|CanvasAds|Interes...,Appended from Pub-Facebook,16-09-2018,Germany,Link,DEU_gaPMG_AZR_AZR_Q1_19_Trial-Social- 39684,21359477,720194,Facebook,3654128,AzurePlatform_FBCTCBrandinMotion-BuyNow_DEU_12...,103350059
4,43356224009494,43356Facebook»DEV-ITP-AZR-FB|CanvasAds|Interes...,Appended from Pub-Facebook,16-09-2018,Germany,Link,DEU_gaPMG_AZR_AZR_Q1_19_Trial-Social- 39684,21359477,720194,Facebook,3654128,AzurePlatform_FBCTCBrandinMotion-BuyNow_DEU_12...,103350059


In [23]:
#new_df.to_csv('new_df.csv',index=False)